測試版

# TTP (Trust Third Party)
#### 幫合約進行解密π
收到 inputZ 與 outputZ，回傳解密+hash結果 input 與 output 給智能合約

In [ ]:
from Crypto.Cipher import AES
from Crypto.Random import get_random_bytes
from Crypto.Util.Padding import pad, unpad

def generate_key():
    return get_random_bytes(32)

def encrypt(data, key):
    cipher = AES.new(key, AES.MODE_CBC)
    ciphertext = cipher.encrypt(pad(data.encode('utf-8'), AES.block_size))
    iv = cipher.iv
    return ciphertext, iv

def decrypt(ciphertext, key, iv):
    cipher = AES.new(key, AES.MODE_CBC, iv)
    decrypted_data = unpad(cipher.decrypt(ciphertext), AES.block_size)
    return decrypted_data.decode('utf-8')

with open("key.txt", "rb") as f:
    key = f.read()

In [ ]:
from web3 import Web3
import time
import json
import base64

ganache_url = "http://localhost:7545"
web3 = Web3(Web3.HTTPProvider(ganache_url))

# 检查连接
if not web3.is_connected():
    print("Unable to connect to Ethereum node.")
    exit()

# 合约地址和 ABI
contract_address = '0x04DAC69F0B1c7b0c0C129Fcb83280c74551550d5'
with open('abi7.json', 'r') as file:
    contract_abi = json.load(file) 

# 创建合约实例
contract = web3.eth.contract(address=contract_address, abi=contract_abi)

# 监听事件
def handle_event(event):
    print(f"Event received: {event}")
    message = event['args']['message']
    ori_data = []
    ori_data.append(event['args']['input1'])
    ori_data.append(event['args']['input2'])
    ori_data.append(event['args']['output'])
    
    # 调用外部程序或 API 获取数据
    data = fetch_data_from_external_source(ori_data)
    # 将数据发送回智能合约
    send_data_to_contract(event['args']['caller'], data)

def fetch_data_from_external_source(ori_data):
    # 模拟外部数据获取过程
    hash_data = []
    for item in ori_data:
        encrypted_data, iv_data = item.strip().split(",")
        decrypted_data = decrypt(base64.b64decode(encrypted_data), key, base64.b64decode(iv_data))
        hash_data.append(web3.solidity_keccak(['string'], [decrypted_data]).hex())
    print(hash_data)
    return hash_data

def send_data_to_contract(caller_address, data):
    # 使用私钥对交易进行签名
    private_key = '0x05bdd1aa1ae4202edd88d25c0c34ae40a1e11ae47f582739e53d67e34ba2284f'
    account_address = '0x09ea376Ff3D21010615c4f263a72AF13AEDF77CD'
    nonce = web3.eth.get_transaction_count(account_address)
    transaction = contract.functions.receiveData(data[0], data[1], data[2]).build_transaction({
        'from': account_address,
        'nonce': nonce,
        'gas': 2000000,
        'gasPrice': web3.to_wei('50', 'gwei')
    })
    signed_txn = web3.eth.account.sign_transaction(transaction, private_key)
    tx_hash = web3.eth.send_raw_transaction(signed_txn.rawTransaction)
    print(f"Transaction sent: {web3.to_hex(tx_hash)}")

# 监听并处理事件
def listen_for_events():
    print("Creating event filter")
    event_filter = contract.events.RequestData.create_filter(fromBlock='latest')
    print("Starting event loop")
    while True:
        #print("Checking for new events")
        for event in event_filter.get_new_entries():
            handle_event(event)
        time.sleep(2)

if __name__ == "__main__":
    
    # 启动事件监听
    listen_for_events()

Creating event filter
Starting event loop
